In [2]:
import os
os.chdir('/media/cj/HAMMERSPACE/Data Science/dsi/capstones/report-card-recession')
import pandas as pd 
import urllib.request 
import chardet

In [6]:
url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/metro/totals/cbsa-est2019-alldata.csv'
urllib.request.urlretrieve(url, 'data/source_files/census/city/2010.csv')
# with open('data/source_files/census/city/2010.csv', encoding = 'UTF-8') as f:
#     print(chardet.detect(f.read()))

schema = {'CSA':str, 'CBSA':str, 'MDIV':str, 'STCOU':str, 'NAME':str, 'LSAD':str, 'POPESTIMATE2010':int, 'POPESTIMATE2011':int, 'POPESTIMATE2012':int, 'POPESTIMATE2013':int, 'POPESTIMATE2014':int, 'POPESTIMATE2015':int, 'POPESTIMATE2016':int, 'POPESTIMATE2017':int, 'POPESTIMATE2018':int, 'POPESTIMATE2019':int}
df = pd.read_csv('data/source_files/census/city/' + '2010.csv', encoding= 'ISO-8859-1', dtype = schema)
drop_cols = []
for column in df.columns:
    if column not in schema.keys():
        drop_cols.append(column)
df.drop(columns = drop_cols, inplace = True)
df

,CSA,CBSA,MDIV,STCOU,NAME,LSAD,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,104,NaN,NaN,NaN,"Albany-Schenectady, NY",Combined Statistical Area,1168901,1169156,1170241,1171171,1171007,1170550,1169639,1171385,1170564,1167594
1,104,10580,NaN,NaN,"Albany-Schenectady-Troy, NY",Metropolitan Statistical Area,871082,872778,874698,877065,878113,879085,879792,882158,882263,880381
2,104,10580,NaN,36001,"Albany County, NY",County or equivalent,304086,304596,305723,306589,307151,307433,307597,307717,306585,305506
3,104,10580,NaN,36083,"Rensselaer County, NY",County or equivalent,159340,159589,159437,159545,159666,159436,159294,159200,159283,158714
4,104,10580,NaN,36091,"Saratoga County, NY",County or equivalent,220109,221111,222483,224087,224513,226078,227122,229276,230170,229863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,566,41400,NaN,39029,"Columbiana County, OH",County or equivalent,107890,107419,106620,105959,105621,104714,103817,103062,102473,101883
1929,566,49660,NaN,NaN,"Youngstown-Warren-Boardman, OH-PA",Metropolitan Statistical Area,564826,562347,558530,555850,553039,548798,544582,541545,538226,536081
1930,566,49660,NaN,39099,"Mahoning County, OH",County or equivalent,238381,237363,235702,234393,233391,231846,230269,229791,229216,228683
1931,566,49660,NaN,39155,"Trumbull County, OH",County or equivalent,209840,208834,207191,206328,204973,203328,201715,200281,198539,197974


In [17]:
from src.constants import TITLE
matches = {}
for index, row in df[df['LSAD'] == 'Combined Statistical Area'].iterrows():
    fips = 'CS' + row['CSA']
    if fips in TITLE.keys():
        matches[fips] = ['CSA', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['CSA', row['NAME'], 'MISSING FROM QCEW']

for index, row in df[df['LSAD'] == 'County or equivalent'].iterrows():
    fips = row['STCOU']
    if fips in TITLE.keys():
        matches[fips] = ['County', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['County', row['NAME'], 'MISSING FROM QCEW']

for index, row in df[df['LSAD'] == 'County or equivalent'].iterrows():
    fips = row['STCOU']
    if fips in TITLE.keys():
        matches[fips] = ['County', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['County', row['NAME'], 'MISSING FROM QCEW']

for index, row in df[df['LSAD'] == 'Metropolitan Division'].iterrows():
    fips = row['MDIV']
    if fips in TITLE.keys():
        matches[fips] = ['Metro Div', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['Metro Div', row['NAME'], 'MISSING FROM QCEW']

for index, row in df[df['LSAD'] == 'Metropolitan Statistical Area'].iterrows():
    fips = 'C' + row['CBSA'][0:4]
    if fips in TITLE.keys():
        matches[fips] = ['Metro SA', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['Metro SA', row['NAME'], 'MISSING FROM QCEW']


for index, row in df[df['LSAD'] == 'Metropolitan Statistical Area'].iterrows():
    fips = 'C' + row['CBSA'][0:4]
    if fips in TITLE.keys():
        matches[fips] = ['Micro SA', row['NAME'], TITLE[fips]]
    else:
        matches[fips] = ['Micro SA', row['NAME'], 'MISSING FROM QCEW']

for area_fips, name in TITLE.items():
    if type(area_fips) == str and area_fips[0:2] != 'US' and area_fips[2:] != 0:
        matches[area_fips] = ['TBD', 'MISSING FROM CENSUS', name]

matching_df = pd.DataFrame.from_dict(matches, orient = 'index', columns = ['Type', 'Name from Census', 'Name from QCEW'])
matching_df



,Type,Name from Census,Name from QCEW
CS104,TBD,MISSING FROM CENSUS,Albany-Schenectady-Amsterdam
CS106,CSA,"Albuquerque-Santa Fe-Las Vegas, NM",MISSING FROM QCEW
CS107,CSA,"Altoona-Huntingdon, PA",MISSING FROM QCEW
CS108,CSA,"Amarillo-Pampa-Borger, TX",MISSING FROM QCEW
CS118,TBD,MISSING FROM CENSUS,Appleton-Oshkosh-Neenah
...,...,...,...
CS512,TBD,MISSING FROM CENSUS,Sioux City-Vermillion
CS516,TBD,MISSING FROM CENSUS,Spartanburg-Gaffney-Union
CS526,TBD,MISSING FROM CENSUS,Sunbury-Lewisburg-Selinsgrove
CS542,TBD,MISSING FROM CENSUS,Union City-Martin


In [14]:
type('d') == str


True